In [1]:
import pandas as pd
from tqdm import tqdm

from sklearn.metrics import mean_absolute_percentage_error as mape
from sklearn.metrics import r2_score

from caml import config
from caml.eio.utils import utils_preprocess_text
from caml.eio.similarity import MLModel

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/bhabalaj/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
annotation_df = pd.read_pickle('../data/6k_grocery_products_annotations.pkl')

In [3]:
naics_df = pd.read_pickle('../data/naics_codes.pkl')

In [4]:
annotation_df = annotation_df.merge(naics_df[['naics_code','eio_co2']], 
                left_on='naics_code', right_on='naics_code', how='left').drop(columns='raw_annotations').drop_duplicates()

In [5]:
annotation_df["text_clean"] = annotation_df["product_text"].apply(lambda x: utils_preprocess_text(x, flg_stemm=False, flg_lemm=True))
annotation_df['text_clean'] = annotation_df['text_clean'].str.replace('\d+', '')
annotation_df['text_clean'] = annotation_df['text_clean'].str.replace('_', ' ', regex=False)
annotation_df['text_clean'] = annotation_df['text_clean'].str.replace('   ', ' ', regex=False)
annotation_df['text_clean'] = annotation_df['text_clean'].str.replace('  ', ' ', regex=False)
annotation_df['text_clean'] = annotation_df['text_clean'].str.lstrip()
annotation_df['text_clean'] = annotation_df['text_clean'].str.rstrip()

/var/folders/rt/c3955xp57xj0gdn078hf2my80000gs/T/ipykernel_8168/200897850.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  annotation_df['text_clean'] = annotation_df['text_clean'].str.replace('\d+', '')


In [ ]:
model = MLModel('all-mpnet-base-v2')
product_list = annotation_df.text_clean.values
naics_list = naics_df.text_clean.values
cosine_scores = model.compute_similarity_scores(product_list, naics_list)

In [ ]:
# Clear the evaluation dataframes where results will be stored.
evaluation_df = pd.DataFrame()
top5_df = pd.DataFrame()

In [ ]:
## Evaluate the products in the test set
# Aggregate the top-20 NAICS descriptions by NAICS codes. Save the top-5. 
for ix in tqdm(range(len(annotation_df))):
    similarity_score = model.rank_similarity_scores(annotation_df, cosine_scores, ix, naics_df)
    aggregated_scores = similarity_score.groupby('naics_code').first()
    aggregated_scores['votes'] = similarity_score.groupby('naics_code').size()
    aggregated_scores = aggregated_scores.sort_values(['cosine_score', 'votes'], ascending=False).reset_index()
    evaluation_df = pd.concat([evaluation_df, aggregated_scores.head(1)])
    top5_df = pd.concat([top5_df, aggregated_scores.head(5)])

In [ ]:
## Compute the top-1 accuracy of the model
# Use annotation_df dataframe if you want to evaluate against all the products at the same time
# instead of just the test set
adf = annotation_df.merge(naics_df[['naics_code','naics_desc']].groupby('naics_code').first(), 
                     left_on='naics_code', right_on='naics_code', how='left')
edf = evaluation_df.merge(naics_df[['naics_code','eio_co2']], left_on='naics_code', 
                          right_on='naics_code', how='left')
if len(edf.product_code.unique()) > len(adf.product_code.unique()):                          
    df = adf.set_index("product_code").join(edf.set_index("product_code"), lsuffix='_human', rsuffix='_model')
else:
    df = edf.set_index("product_code").join(adf.set_index("product_code"), lsuffix='_human', rsuffix='_model')
rf = df[df.naics_code_human == df.naics_code_model]

print("Top-1 accuracy w.r.t NAICS codes: ", len(rf.index.unique())/len(df.index.unique()))
print("Correct predictions: {}, Total Products: {}".format(len(rf.index.unique()), len(df.index.unique())))

## Compute the top-1 accuracy with respect to BEA codes (top 4 digits of NAICS code)
df.bea_human = df.naics_code_human//100
df.bea_model = df.naics_code_model//100
bf = df[df.bea_human == df.bea_model]
print("Top-1 accuracy w.r.t BEA codes: ", len(bf.index.unique())/len(df.index.unique()))
print("Correct predictions: {}, Total Products: {}".format(len(bf.index.unique()), len(df.index.unique())))

# Compute the mean absolute percentage error and R^2 value w.r.t EIF for zero-shot prediction
df = df.dropna()
y_true = df.groupby(df.index).first().eio_co2_human
y_pred = df.groupby(df.index).first().eio_co2_model
print("The mean absolute percentage error for predicting kgCO2e/$: ", mape(y_true, y_pred))
print("The R2 value for predicting kgCO2e/$: ", r2_score(y_true, y_pred))

In [ ]:
full_df = annotation_df.merge(top5_df, left_on='product_code', right_on='product_code', how='left')
print(full_df.shape)
full_df.head()

In [ ]:
full_df['label'] = (full_df.naics_code_x == full_df.naics_code_y).astype('float')
full_df = full_df[['product_code','product_text','naics_code_x', 'naics_desc', 'label']].dropna()
full_df.head()

In [ ]:
train_df = full_df.iloc[int(0.25*len(full_df)):]
test_df = full_df.iloc[:int(0.25*len(full_df))]
train_df.shape, test_df.shape

In [ ]:
model.fine_tune(train_df)

In [ ]:
eval_ft_df = pd.DataFrame()

In [ ]:
test_df = test_df[test_df.label == 1]
test_df = test_df.groupby('product_code').first().reset_index()
test_df = test_df.rename(columns={'product_code': 'parent_product', 
                                  'product_text': 'text_clean', 
                                  'naics_code_x': 'naics_code'})

naics_list = naics_df.text_clean.to_list()
product_list = test_df.text_clean.values
cosine_scores = model.compute_similarity_scores(product_list, naics_list)

In [ ]:
for ix in tqdm(range(len(test_df))):
    similarity_score = model.rank_similarity_scores(test_df, cosine_scores, ix, naics_df)
    aggregated_scores = similarity_score.groupby('naics_code').first()
    aggregated_scores['votes'] = similarity_score.groupby('naics_code').size()
    aggregated_scores = aggregated_scores.sort_values(['cosine_score', 'votes'], ascending=False).reset_index()
    eval_ft_df = pd.concat([eval_ft_df, aggregated_scores.head(1)])

In [ ]:
## Compute the top-1 accuracy of the model
# Use annotation_df dataframe if you want to evaluate against all the products at the same time
# instead of just the test set
adf = annotation_df.merge(naics_df[['naics_code','naics_desc']].groupby('naics_code').first(), 
                     left_on='naics_code', right_on='naics_code', how='left')
edf = eval_ft_df.merge(naics_df[['naics_code','eio_co2']], left_on='naics_code', 
                          right_on='naics_code', how='left')
if len(edf.product_code.unique()) > len(adf.product_code.unique()):                          
    df = adf.set_index("product_code").join(edf.set_index("product_code"), lsuffix='_human', rsuffix='_model')
else:
    df = edf.set_index("product_code").join(adf.set_index("product_code"), lsuffix='_human', rsuffix='_model')
rf = df[df.naics_code_human == df.naics_code_model]

print("Top-1 accuracy w.r.t NAICS codes: ", len(rf.index.unique())/len(df.index.unique()))
print("Correct predictions: {}, Total Products: {}".format(len(rf.index.unique()), len(df.index.unique())))

## Compute the top-1 accuracy with respect to BEA codes (top 4 digits of NAICS code)
df.bea_human = df.naics_code_human//100
df.bea_model = df.naics_code_model//100
bf = df[df.bea_human == df.bea_model]
print("Top-1 accuracy w.r.t BEA codes: ", len(bf.index.unique())/len(df.index.unique()))
print("Correct predictions: {}, Total Products: {}".format(len(bf.index.unique()), len(df.index.unique())))

# Compute the mean absolute percentage error and R^2 value w.r.t EIF for zero-shot prediction
df = df.dropna()
y_true = df.groupby(df.index).first().eio_co2_human
y_pred = df.groupby(df.index).first().eio_co2_model
print("The mean absolute percentage error for predicting kgCO2e/$: ", mape(y_true, y_pred))
print("The R2 value for predicting kgCO2e/$: ", r2_score(y_true, y_pred))